In [30]:
import cudf
import cupy as cp
import numpy as np
import sys
import time
import numba
from numba import cuda

from cupy import prof

In [31]:
intiSize = int(1e8)
interval = int(2e7)
numCols = 3
iterations=1

enableDF=True
enableMultiDF=False
enableCUPY=True

In [32]:
import rmm

def rmm_cupy_allocator(nbytes):
    """
    A CuPy allocator that makes use of RMM.
Examples
    --------
    >>> import rmm
    >>> import cupy
    >>> cupy.cuda.set_allocator(rmm.rmm_cupy_allocator)
    """
    if cupy is None:
        raise ModuleNotFoundError("No module named 'cupy'")
    buf = librmm.device_buffer.DeviceBuffer(size=nbytes)
    dev_id = -1 if buf.ptr else cupy.cuda.device.get_device_id()
    mem = cupy.cuda.UnownedMemory(
        ptr=buf.ptr, size=buf.size, owner=buf, device_id=dev_id
    )
    ptr = cp.cuda.memory.MemoryPointer(mem, 0)
    return ptr

cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

In [33]:
size = intiSize

df=cudf.DataFrame()
sortedDF=cudf.DataFrame()

cp.random.seed(234234)

strCol = []
for c in range(numCols):
    strCol.append(str(c))
# print(strCol)

for c in range(numCols):
    randvals = cp.random.randint(low= 0, high=2**20, size=size, dtype=cp.int32)

    randvals = cp.asfortranarray(randvals)
    dfCol = cudf.from_dlpack(randvals.toDlpack())
    
    df.add_column(str(c), dfCol)

# print(df)


In [34]:


if(enableDF==True):
    with prof.time_range("cudf-sort", 0):
        cuda.synchronize()
        dfstart = time.time()
        for it in range(iterations):
    #         sortedDF = df.sort_values(["0","1","2","3","4"])
            sortedDF = df.sort_values(strCol)
        cuda.synchronize()
        dfstop  = time.time()


    print('%.2f'%((dfstop-dfstart)/(float(iterations))),end=", ")
    print('%.2f'%(float(iterations*size)/(dfstop-dfstart)/1e6),end=", ")
    print("")

# print(sortedPerm)
# print(sortedDF)

0.86, 115.93, 


In [35]:
multiDF=cudf.DataFrame()
unsorted = cudf.DataFrame()
size = intiSize

# print(unsorted['index'])

interest = 3
if (enableMultiDF==True):
    with prof.time_range("df-multi-sort", 0):
        cuda.synchronize()
        mldfstart = time.time()
        for it in range(iterations):
        #     unsorted['index']=[ int(i) for i in range(intiSize)]
            indexArr = cp.arange(intiSize)
            indexArr = cp.asfortranarray(indexArr)
            unsorted['index'] = cudf.from_dlpack(indexArr.toDlpack())
            unsorted['index'].head()    
            for c1 in range(numCols):
                c=numCols-c1-1
            #     print(c)
                unsorted['data'] = df[str(c)].iloc[unsorted['index']]
        #         if(c==interest):
        #             print(unsorted)

            #     indices = unsorted['index']
            #     unsorted = unsorted.iloc[indices]
            #     if(c==interest):
            #         print(unsorted)    

                unsorted = unsorted.sort_values('data')
        #         if(c==interest):
        #             print(unsorted)

        cuda.synchronize()
        mldfstop  = time.time()


    print('%.2f'%((mldfstop-mldfstart)/(float(iterations))),end=", ")
    print('%.2f'%(float(iterations*size)/(mldfstop-mldfstart)/1e6),end=", ")
    print("")
    multiDF = df.iloc[unsorted['index']]
    print(multiDF)

    print(multiDF.equals(sortedDF))


In [36]:
# diffDF = cudf.DataFrame()
# diffDF = smartDF.equals(sortedDF)
# print(diffDF)

In [37]:
multiPyDF=cudf.DataFrame()
unsorted = cudf.DataFrame()
size = intiSize
# unsorted['index']=[ int(i) for i in range(intiSize)]


# print(unsorted['index'])

interest=3

if(enableCUPY==True):
    with prof.time_range("multi-sort", 0):

        cuda.synchronize()
        mldfstart = time.time()
        for it in range(iterations):

            indexArr = cp.arange(intiSize)

            for c1 in range(numCols):
                c=numCols-c1-1
            #     print(c)
        #         dataArray = cp.asarray(df[str(c)].iloc[indexArr])
                dataArray = cp.asarray(df[str(c)])
                dataArray = dataArray[indexArr]
        #         if(c==interest):
        #             print(dataArray)
        #             print(indexArr)

                newIndex = cp.argsort(dataArray)
                indexArr = indexArr[newIndex]
            #####        unsorted = unsorted.sort_values('data')
        #         if(c==interest):
        #             print(dataArray[indexArr])
        #             print(indexArr)

        cuda.synchronize()
        mldfstop  = time.time()
        # print(indexArr)

    # print(dataArray[indexArr])
    # print(cp.asarray(df[str(1)].iloc[indexArr]))


    print('%.2f'%((mldfstop-mldfstart)/(float(iterations))),end=", ")
    print('%.2f'%(float(iterations*size)/(mldfstop-mldfstart)/1e6),end=", ")
    print("")

#     multiPyDF = df.iloc[indexArr]
#     print(multiPyDF)
#     print(multiPyDF.equals(sortedDF))


0.30, 331.13, 


In [38]:
multiPyDF = df.iloc[indexArr]
print(multiPyDF)

print(multiPyDF.equals(multiDF))
# print(newUnsorted.iloc[newUnsorted['index']])


                0        1       2
75629383        0     9094  514397
55366960        0     9224  287479
62985407        0    10932  327524
14556757        0    14267   54170
41708702        0    18705  275136
...           ...      ...     ...
88763613  1048575  1016785  638180
21552791  1048575  1021259  641297
29929935  1048575  1023611  693230
33038233  1048575  1030260  975646
77239708  1048575  1041066  407941

[100000000 rows x 3 columns]
False
